In [33]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
import nltk
from string import punctuation
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB



data = pd.read_csv('SMSSpamCollection.tsv',sep='\t',names = ['type','msg'])
stemmer = PorterStemmer()
ab = ["i've", "i'll", "we'll", "'ve"]


def text_process(t):
    all_words = []
    t = t.replace('.','. ').replace('!','! ').replace('?','? ')
    for sent in nltk.sent_tokenize(t.lower()):
        words = nltk.word_tokenize(sent)        
        words = [word for word in words if (word not in stopwords.words('english')) and (word not in ab)]        
        words = [stemmer.stem(word) for word in words]
        words = [word for word in words if word not in punctuation]
        words = [word for word in words if not word.isnumeric()]
        all_words += words    
    all_words = ''.join([ch for ch in ' '.join(all_words) if (ch not in punctuation) and (not ch.isnumeric())])    
    return all_words


tfidfV = TfidfVectorizer()
tf_idf = tfidfV.fit_transform(data['msg'].apply(text_process))
tf_df = pd.DataFrame(tf_idf.toarray(),columns = tfidfV.get_feature_names())
xtrain, xtest, ytrain, ytest = train_test_split(tf_df, data['type'], test_size=0.25, random_state=101)
model_bnb = BernoulliNB().fit(xtrain,ytrain)
print(model_bnb.score(xtest,ytest))


0.9691091954022989
